In [1]:
# import random as rnd
# import secrets as scr

from datetime import datetime
from random import SystemRandom

rnd = SystemRandom()


class Lotteria:
    def __init__(self,
                 max_numbers=90,
                 max_extra=90,
                 length=6,
                 extra=1):
        self.max_numbers = max_numbers
        self.max_extra = max_extra
        self.length = length
        self.extra = extra

    @staticmethod
    def pallottoliere(num):
        while True:
            yield rnd.randint(1, num)

    @staticmethod
    def choice(length, num):
        def extraction():
            rnd.shuffle(numbers := list(range(1, num+1)))
            single = rnd.choice(numbers)
            numbers.remove(single)

            return single

        return [extraction() for _ in range(length)]

    @staticmethod
    def sample(length, num):
        numbers = list(range(1, num+1))

        return rnd.sample(numbers, k=length)

    def int(self, length, num):
        combo = set()
        while len(combo) < length:
            combo.add(next(self.pallottoliere(num)))

        return combo

    @staticmethod
    def many_(backend, length, num):
        samples = [backend(length, num) for _ in range(num)]
        rnd.shuffle(samples)

        return rnd.choice(samples)

    @property
    def backend(self):
        return self._backend

    @property
    def many(self):
        return self._many

    def __call__(self, backend='sample', many=True):
        self._backend = backend
        self._many = many

        backend = eval(backend,
                       {'__builtins__': {}},
                       {'choice': self.choice, 'int': self.int, 'sample': self.sample})

        if self._many:
            combo = self.many_(backend,
                               self.length,
                               self.max_numbers)

            if self.extra:
                super = self.many_(backend,
                                   self.extra,
                                   self.max_extra)

        else:
            combo = backend(self.length, self.max_numbers)

            if self.extra:
                super = backend(self.extra, self.max_extra)

        now = datetime.now().strftime("%d/%m/%Y %H:%M")

        print(f'Estrazione del: {now}', '\nnumbers combo:', *sorted(combo))
        if self.extra:
            print('Superstar:', *sorted(super))  # type: ignore

In [2]:
eurojack = Lotteria(max_numbers=50, max_extra=12, length=5, extra=2)

In [3]:
for b in ['choice', 'int', 'sample']:
    print('######')
    eurojack(backend=b, many=False)
    print('backend:', eurojack.backend)

print('\n################# many ####################\n')

for b in ['choice', 'int', 'sample']:
    print('######')
    eurojack(backend=b, many=True)
    print('backend:', eurojack.backend)

######
Estrazione del: 04/06/2023 10:37 
numbers combo: 20 26 37 46 50
Superstar: 1 4
backend: choice
######
Estrazione del: 04/06/2023 10:37 
numbers combo: 2 14 23 35 37
Superstar: 4 11
backend: int
######
Estrazione del: 04/06/2023 10:37 
numbers combo: 2 18 23 31 48
Superstar: 4 8
backend: sample

################# many ####################

######
Estrazione del: 04/06/2023 10:37 
numbers combo: 4 8 9 41 44
Superstar: 6 8
backend: choice
######
Estrazione del: 04/06/2023 10:37 
numbers combo: 1 25 28 34 35
Superstar: 9 12
backend: int
######
Estrazione del: 04/06/2023 10:37 
numbers combo: 4 28 35 39 49
Superstar: 2 8
backend: sample


In [4]:
superenalotto = Lotteria()

In [5]:
superenalotto()

Estrazione del: 04/06/2023 10:37 
numbers combo: 32 53 61 64 83 86
Superstar: 20


In [6]:
superenalotto(backend='choice')

Estrazione del: 04/06/2023 10:37 
numbers combo: 2 7 13 29 65 84
Superstar: 2


In [7]:
superenalotto.many

True

In [8]:
winForLife = Lotteria(20, 20, 10, 1)
winForLife(backend='int', many=False)

Estrazione del: 04/06/2023 10:38 
numbers combo: 2 3 4 5 6 10 13 14 16 19
Superstar: 1


In [9]:
# trying to set backend via attribute: fails as expected
winForLife.backend = 'choice'

AttributeError: property 'backend' of 'Lotteria' object has no setter